# ADS 509 Module 1: APIs and Web Scraping

This notebook has three parts. In the first part you will pull data from the Twitter API. In the second, you will scrape lyrics from AZLyrics.com. In the last part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the some of the user information for the followers of your artist and store them in text files. 


# Notebook Setup

In [1]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter

In [2]:
# Use this cell for any import statements you add
import pandas as pd
import shutil
import time
import tqdm

def timer(length_of_time):
    timer = round(length_of_time / 60)

    for num in range(0,timer,1):
        time.sleep(60)
        print(f'Sequence {num} complete out of {timer}.')
        
    print('Ring Ring Ring')
    
def folder_maker(folder2make):
    # create directory to store html files
    if os.path.isdir(folder2make):
        shutil.rmtree(folder2make)
    os.mkdir(folder2make)


# Twitter API Pull

We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

In [3]:
from api_keys import api_key, api_key_secret, bearer_token

def client_FN():
    return tweepy.Client(bearer_token,wait_on_rate_limit=True)

# Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull some of the followers @37chandler.
1. Explore response data, which gives us information about Twitter users. 
1. Pull the last few tweets by @37chandler.


In [4]:
client = client_FN()

handle = "37chandler"
chandler_user_data = client.get_user(username=handle)

chandler_followers = client.get_users_followers(
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    chandler_user_data.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)

Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users. 

In [5]:
num_to_print = 3

for idx, user in enumerate(chandler_followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.\n")
    
    if idx >= (num_to_print - 1) :
        break

Dave Renn lists 'None' as their location.
 Following: 42, Followers: 11.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.



Let's find the person who follows this handle who has the most followers. 

In [6]:
max_followers = 0

for idx, user in enumerate(chandler_followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

print(max_follower_user)
print(max_follower_user.public_metrics)

WedgeLIVE
{'followers_count': 14157, 'following_count': 2223, 'tweet_count': 56071, 'listed_count': 218}


Let's pull some more user fields and take a look at them. The fields can be specified in the `user_fields` argument. 

In [7]:
# response = client.get_user(id=chandler_user_data.data.id,
#                           user_fields=["created_at","description","location",
#                                        "entities","name","pinned_tweet_id","profile_image_url",
#                                        "verified","public_metrics"])

# for i, (field, value) in enumerate(response.data.items()):
#     print(f"for {field} we have {value}")
# print(f"\n\nThere are a total of {i+1} fields!")

--- 

Now a few questions for you about the user object.

<u>Q: How many fields are being returned in this user object?</u>

A: There is a total of 9 fields.

---

<u>Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.)</u>

A: Public Metrics is a Dict containing scalar-values, profile_image_url contains a url string, date time as "created_at", and last but not least "description" is also a string.

---
<u>Q: How many friends, followers, and tweets does this user have? </u>

A:<br>
followers count: 194<br>
following count: 590<br>
tweet count: 989<br>
listed count: 3



Although you won't need it for this assignment, individual tweets can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the fields that are available about Tweets.

In [8]:
# response = client.get_users_tweets(chandler_user_data.data.id)

# # By default, only the ID and text fields of each Tweet will be returned
# for idx, tweet in enumerate(response.data) :
#     print(tweet.id)
#     print(tweet.text)
#     print()
    
#     if idx > 5:
#         break

## Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using `client.get_users_followers`. This function has a parameter, `max_results`, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the _pagination_ of our results. 

The return object has the `.data` field, where the results will be found. It also has `.meta`, which we use to select the next "page" in the results using the `next_token` result. I will illustrate the ideas using our user from above. 


### Rate Limiting

Twitter limits the rates at which we can pull data, as detailed in [this guide](https://developer.twitter.com/en/docs/twitter-api/rate-limits). We can make 15 user requests per 15 minutes, meaning that we can pull $4 \cdot 15 \cdot 1000 = 60000$ users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of `handle`.  


In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.) 

In [9]:
# handle_followers = []
# pulls = 0
# max_pulls = 100
# next_token = None
# max_results = 2 # when you do this for real, set this to 1000!

# while True :

#     followers = client.get_users_followers(
#         chandler_user_data.data.id, 
#         max_results=max_results, 
#         pagination_token = next_token,
#         user_fields=needed_fields
#     )
#     pulls += 1
    
#     for follower in followers.data : 
#         follower_row = (follower.id,follower.name,follower.created_at,follower.description)
#         handle_followers.append(follower_row)
    
#     if 'next_token' in followers.meta and pulls < max_pulls :
#         next_token = followers.meta['next_token']
#     else : 
#         break

# handle_followers[2]

Now let's take a look at your artists and see how long it is going to take to pull all their followers. 

In [10]:
artists = dict()

handles = ['FFDP','OfficialRezz']

client = client_FN()
for handle in handles: 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")

It would take 11.27 hours to pull all 675974 followers for FFDP. 
It would take 5.02 hours to pull all 301231 followers for OfficialRezz. 


Depending on what you see in the display above, you may want to limit how many followers you pull. It'd be great to get at least 200,000 per artist. 

As we pull data for each artist we will write their data to a folder called "twitter", so we will make that folder if needed.

In [11]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

# checks to see if current dir has folder named twitter
if os.path.isdir("twitter"): 
    shutil.rmtree("twitter/")

os.mkdir("twitter")

In the following cells, build on the above code to pull some of the followers and their data for your two artists. As you pull the data, write the follower ids to a file called `[artist name]_followers.txt` in the "twitter" folder. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

Extract and store the following fields: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

 Store the fields with one user per row in a tab-delimited text file with the name `[artist name]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 


One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. I've included some example code to do that below the stub. 

In [20]:
handles = ['FFDP']
handles[0]

'FFDP'

In [21]:
client = client_FN()
client.get_user(username=handles[0])


Response(data=<User id=44984488 name=Five Finger Death Punch username=FFDP>, includes={}, errors=[], meta={})

In [22]:
max_results = 1000
max_pulls = 100
next_token=None
pulls = 0
# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

num_followers_to_pull = 200*1000

for handle in handles:
    print(f'Processing {handle}')
    client = client_FN()
    user_data = client.get_user(username=handle)
    
    follower_data = []
    follower_ids = []
    current_follower_count = 0
    
    needed_fields = ['description',
                     'id',
                     'location',
                     'name',
                     'public_metrics']

    while True :
        try:
            current_follower_count += max_results
            
            # print status/current count during pagination/pause
            if current_follower_count % 15000 == 0:
                print(f'Current Count: {current_follower_count}\n')
            # restablish connection via new connection
            client = client_FN()

            followers = client.get_users_followers(
                user_data.data.id, 
                max_results=max_results, 
                pagination_token=next_token,
                user_fields=needed_fields
            )
            pulls += 1
            

# --------------------- OLD
#             for follower in followers.data : 
#                 follower_row = (handle,
#                                 follower.id,
#                                 follower.name,
#                                 follower.username, # screen_name
#                                 follower.description,
#                                 follower.location,
#                                 follower.public_metrics['followers_count'])
#                 handle_followers.append(follower_row)

# --------------------- NEW
            for follower in followers.data:
                follower_ids.append(follower.id)

                follower_row = {'Artist':handle,
                                'Id':follower.id,
                                'Name':follower.name,
                                'User_Name':follower.username, # screen_name
                                'Description':follower.description,
                                'Location':follower.location,
                                'Followers_Count':follower.public_metrics['followers_count']}
                follower_data.append(follower_row)

                
            if 'next_token' in followers.meta and pulls < max_pulls :
                next_token = followers.meta['next_token']
            # If num of followers pulled reaches designated limit num_followers_to_pull
            elif current_follower_count > num_followers_to_pull:
                break
            else : 
                break
        except:
            pass
    
    # Write the data to the output file in the `twitter` folder.
    pd.DataFrame(follower_data).to_csv(f'twitter/{handle}_followers_data.txt', index=False)
    
    pd.Series({handle:follower_ids}).to_csv(f'twitter/{handle}_followers.txt', index=False)

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)

Processing FFDP
Current Count: 15000



Rate limit exceeded. Sleeping for 888 seconds.


Current Count: 30000



Rate limit exceeded. Sleeping for 888 seconds.


Current Count: 45000



...

Current Count: 75000



Rate limit exceeded. Sleeping for 887 seconds.


Current Count: 90000



Rate limit exceeded. Sleeping for 888 seconds.


Current Count: 105000

1:32:33.683915


#### Check Saved Data

In [26]:
view = pd.read_csv('twitter/FFDP_followers_data.txt')
print(f'DF Shape {view.shape}')
view.head(2)

DF Shape (99997, 7)


,Artist,Id,Name,User_Name,Description,Location,Followers_Count
0,FFDP,1568772872851570691,neil,neil6942,NaN,NaN,1
1,FFDP,1187816796012855304,Pixel,SnootBoopPup,A pup who loves to boop the snoot! | 27 | Pans...,"South Wales, United Kingdom",2588


In [28]:
view = pd.read_csv('twitter/OfficialRezz_followers_data.txt')
print(f'DF Shape {view.shape}')
view.head(2)

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


The file for Rezz was unreadable to Pandas, but readable to MacOS's Numbers software, which I loaded the text file into, tell the software the deliminator type and exported the file as a CSV which is used in the cell above.

In [ ]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description)
clean_description

---

# Evaluation

This assignment asks you to pull data from the Twitter API and scrape www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [ ]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

---

## Checking Twitter Data

The output from your Twitter API pull should be two files per artist, stored in files with formats like `cher_followers.txt` (a list of all follower IDs you pulled) and `cher_followers_data.txt`. These files should be in a folder named `twitter` within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [4]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: FFDP and OfficialRezz.


In [6]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_data_file,'r') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")
    

We see 122323 in your follower file for FFDP, assuming a header row.


FileNotFoundError: [Errno 2] No such file or directory: 'twitter/FFDP_followers_data.txt'

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [ ]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")
